In [ ]:
%cd ../

In [ ]:
import plasma.parallel_processing as pp
import time

In [ ]:
def compute(data):
    time.sleep(1)
    return data * 2

def print_pass(data):
    print(f'{data / 2**5}')
    return data

In [ ]:
class Outer(pp.communicators.Flow):

    def __init__(self):
        super().__init__()
        manager = pp.mp.Manager()

        self.compute1 = Inner()
        self.compute2 = compute
        self.compute3 = Inner()
        self.printer = print_pass
        self.aggregator = pp.communicators.Aggregator(20, 0.5, manager)

        self.registerIOs(
            compute1={
                'compute1': pp.queues.ProcessQueue(2),
                'compute2': pp.queues.ProcessQueue(2),
            },
            compute2=pp.queues.ProcessQueue(2),
            compute3={
                'compute1': pp.queues.ProcessQueue(2),
                'compute2': pp.queues.ProcessQueue(2),
            },
            printer=pp.queues.ProcessQueue(),
            aggregator=pp.queues.ProcessQueue(),
        )


class Inner(pp.communicators.Flow):

    def __init__(self):
        super().__init__()

        self.compute1 = compute
        self.compute2 = compute

caller = Outer()
caller

In [ ]:
with caller:
    for i in range(20):
        caller.put(i)
    
    print(caller.aggregator.wait())